# 画像をimg_comparison_tool用に配置するためのスクリプト

In [1]:
original_dir = "/home/usrs/fukuoka/program/research/data/kodak"
a_dir = "/home/usrs/fukuoka/research/spatially_adaptive_realism/result/exp0-3-8_2_nogan_nomse_tanh/kodak/iter500k"
method_a_name = "exp0-3-8_2_nogan_nomse_tanh"
b_dir = "/home/usrs/fukuoka/research/spatially_adaptive_realism/result/exp0-2-14/kodak"
method_b_name = "exp0-2-14"
ex_name = "20240924_exp0-3-8_2_nogan_nomse_tanh"
size=24
ext = ".png"

from glob import glob
import os
import shutil
import numpy as np
import pandas as pd
import cv2

original_path_list = glob(os.path.join(original_dir, f"*{ext}"))
a_path_list = glob(os.path.join(a_dir, f"*{ext}"))
b_path_list = glob(os.path.join(b_dir, f"*{ext}"))
# print(original_path_list)
# print(a_path_list)
# print(b_path_list)

In [ ]:
# 必要ない画像を除外する
def exclude_image(path_list):
    res = []
    for p in path_list:
        img_name = os.path.basename(p)
        if "tmap" not in img_name:
            res.append(p)
    return res

In [ ]:
a_path_list = exclude_image(a_path_list)
b_path_list = exclude_image(b_path_list)
original_path_list = exclude_image(original_path_list)

assert len(original_path_list) == len(a_path_list) == len(b_path_list)

In [2]:
out_dir = os.path.join("./images/", ex_name)

dir = os.path.join("./images/", ex_name)
 
if not os.path.exists(dir):
    # ディレクトリが存在しない場合、ディレクトリを作成する
    os.makedirs(dir)

for i in range(1, size+1):
    p = os.path.join(dir, f"img{i}")
    # print(p)
    if not os.path.exists(p):
        os.makedirs(p)


In [3]:
img_info_list = []

for i in range(size):
    a_img_name = a_path_list[i].split('/')[-1]
    b_img_name = b_path_list[i].split('/')[-1]
    original_img_name = original_path_list[i].split('/')[-1]
    assert a_img_name == b_img_name and b_img_name == original_img_name, "method aとmethod bと元画像の画像ファイル名は全て同じである必要があります"

path_lists = [a_path_list, b_path_list]
map = ["a", "b"]
for i in range(size):
    ts = [1, 2]
    img_name = a_path_list[i].split('/')[-1].split('.')[-2]
    img_info = {"img_id":f"img{i+1}", "img_name":img_name}
    
    # 元画像コピー
    shutil.copy(original_path_list[i], os.path.join(out_dir, f"img{i+1}/3.jpg"))
    
    #手法aと手法bのスワップ
    r = np.random.rand()
    if r >= 0.5:
        ts[1], ts[0] = ts
        img_info["1.jpg(methodA)"] = method_b_name
        img_info["2.jpg(methodB)"] = method_a_name
    else:
        img_info["1.jpg(methodA)"] = method_a_name
        img_info["2.jpg(methodB)"] = method_b_name

    for j in range(2):
        out_path = os.path.join(out_dir, f"img{i+1}/{ts[j]}.jpg")
        # print(out_path)
        shutil.copy(path_lists[j][i], out_path)
    img_info_list.append(img_info)
    
# ログに記録
img_info_df = pd.DataFrame(img_info_list)
# print(img_info_df)
img_info_df.to_csv(os.path.join("./images/", ex_name, "img_info.csv"))

AssertionError: method aとmethod bと元画像の画像ファイル名は全て同じである必要があります

In [ ]:
path_list = glob(f"./images/{ex_name}/img*")

for p in path_list:
    dir_id  = p.split("/")[-1][3:]
    if os.path.isdir(p):
        shutil.copytree(p, f"./images/{ex_name}/img{dir_id}_large")
        
for i in range(1, size+1):
    path_list = glob(f"./images/{ex_name}/img{i}_large/*.jpg")
    for p in path_list:
        # print(p)
        img = cv2.imread(p)
        # print(img.shape)
        h, w, _ = img.shape
        img_resize = cv2.resize(img, None, fx=2.0, fy=2.0)
        # print(img_resize.shape)
        cv2.imwrite(p, img_resize)